In [1]:
from data.load import load_train_data, load_test_data
from model.convolutional import build_compile_fit, reshape_input

In [2]:
x_train, y_train = load_train_data()
x_test, y_test = load_test_data()

x_train_reshaped, y_train_reshaped = reshape_input(x_train, y_train)
x_test_reshaped, y_test_reshaped = reshape_input(x_test, y_test)

In [3]:
model, history = build_compile_fit(x_train_reshaped, y_train_reshaped, epochs=10)
model.summary()

Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2097 - accuracy: 0.9375

In [4]:
from data.file import model_convolutional

model.save(model_convolutional)


INFO:tensorflow:Assets written to: /Users/raffael/learn/fhnw/xai/xai-adversarial-attack/data/model/convolutional/assets
